# Operations Reasearch: Examples for Gurobipy

This code is for students who take the class Operations Research.
Students should finish the installation of Gurobi and Python before started and make sure an academic liscense for Gurobi is applied and activated.

We introduce an example for linear programming problem in order to help students understand how to solve optimization programs with codes.
More insturction is provided in the lecture video.

## Producing desks and tables

Consider the problem we introduced in Operations Research: Modeling and Application, we have

\begin{split}
    \begin{array}{r}
        \max \\ \mbox{s.t.} \\ \\ \\ \\ \\ 
    \end{array} &
    \begin{array}{rcrcll}
        700x_1 & + & 900x_2 & & & \\ 
        3x_1 & + & 5x_2 & \leq & 3600\quad\! & \mbox{(wood)} \\		
        x_1 & + & 2x_2 & \leq & 1600\quad\! & \mbox{(labor)} \\		
        50x_1 & + & 20x_2 & \leq & 48000\:\ & \mbox{(machine)} \\
        x_1 & & & \geq & 0
        \\
        & & x_2 & \geq & 0.
    \end{array}
\end{split}

Let's consturct our problem with Gurobi step by step.

 We should import the Gurobi optimization package called **gurobipy** first.

In [1]:
from gurobipy import *

Use Gurobi functions **Model** and **addVar** to build a new model and add variables. Remember to set proper parameters as function inputs. In **addVar**, use **lb** to set the lower bound, **vtype** to set the type, and **name** to set the name of a variable.

Besides, the paramter value **GRB.CONTINUOUS** means the variable is a continuous number.

In [2]:
# build a new model
eg1 = Model("eg1")  # build a new model, name it as "eg1"

x1 = eg1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'x1')
x2 = eg1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x2")

Using license file C:\Users\user\gurobi.lic
Academic license - for non-commercial use only


Use function **setObjective** and **addConstr** to set the objective function and add some constraints. It's necessary to set whether a problem is a maximization or minimization program. Also, remember to give all constraints, variables and the model distinct names.

In [3]:
# setting the objective function
# use GRB.MAXIMIZE for a maximization problem
eg1.setObjective(700 * x1 + 900 * x2, GRB.MAXIMIZE) 

# add constraints and name them
eg1.addConstr(3 * x1 + 5 * x2 <= 3600, "resource_wood")
eg1.addConstr(x1 + 2 * x2 <= 1600, "resource_labor")
eg1.addConstr(50 * x1 + 20 * x2 <= 48000, "resource_machine")

<gurobi.Constr *Awaiting Model Update*>

Use **optimize** to run and solve the model. Finally, we can use **getVars** to list all of the variables and use **objVal** to get the objective value.

In [4]:
eg1.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xa395d65c
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [7e+02, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 5e+04]
Presolve time: 0.02s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+32   3.593750e+30   2.000000e+02      0s
       3    7.8947368e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.04 seconds
Optimal objective  7.894736842e+05


In [5]:
for var in eg1.getVars():
    print(var.varName, '=', var.x)
print("objective value =", eg1.objVal)

x1 = 884.2105263157895
x2 = 189.47368421052633
objective value = 789473.6842105263


Now let's try to decoupling the data from the model. The data part is as below:

In [6]:
products = range(2)  # 2 products    
resources = range(3)  # 3 resources

prices = [700, 900]    
resource_consumptions = [[3 , 5 ],
                         [1 , 2 ],
                         [50, 20]]
resource_limitations = [3600, 1600, 48000]

We can rewrite our model in a simpler way.

In [7]:
eg1_decoupling = Model("eg1_decoupling")

x = []
for i in products:
    x.append(eg1_decoupling.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'x' + str(i)))

eg1_decoupling.setObjective(quicksum(prices[i] * x[i] for i in products) , GRB.MAXIMIZE) 

# add constraints and name them
eg1_decoupling.addConstrs((quicksum(resource_consumptions[j][i] * x[i] for i in products) 
                           <= resource_limitations[j] for j in resources), "Resource_limitation")

eg1_decoupling.optimize()

for var in eg1_decoupling.getVars():
    print(var.varName, '=', var.x)
print("objective value =", eg1_decoupling.objVal)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xa395d65c
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [7e+02, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 5e+04]
Presolve time: 0.01s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+32   3.593750e+30   2.000000e+02      0s
       3    7.8947368e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  7.894736842e+05
x0 = 884.2105263157895
x1 = 189.47368421052633
objective value = 789473.6842105263
